In [14]:
# 1) Install deps (first time only)
!pip install pandas openpyxl kaggle --quiet

# 2) (Optional) Kaggle CLI setup if you want to download via code:
# Place kaggle.json to ~/.kaggle/kaggle.json with your Kaggle API credentials.

# 3) Convert XLSX → CSV
import pandas as pd

xlsx_path = "./storedata_total.xlsx"          # <-- downloaded Kaggle XLSX filename
csv_path = "./dataset1.csv"
df = pd.read_excel(xlsx_path, engine="openpyxl")
df.to_csv(csv_path, index=False)
print("Rows, Cols:", df.shape)
df.head()

/opt/conda/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Rows, Cols: (30801, 15)


,custid,retained,created,firstorder,lastorder,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,favday,city
0,6H6T6N,0,2012-09-28,2013-08-11 00:00:00,2013-08-11 00:00:00,29,100.000000,3.448276,14.52,0.000000,0,0,0,Monday,DEL
1,APCENR,1,2010-12-19,2011-04-01 00:00:00,2014-01-19 00:00:00,95,92.631579,10.526316,83.69,0.181641,1,1,1,Friday,DEL
2,7UP6MS,0,2010-10-03,2010-12-01 00:00:00,2011-07-06 00:00:00,0,0.000000,0.000000,33.58,0.059908,0,0,0,Wednesday,DEL
3,7ZEW8G,0,2010-10-22,2011-03-28 00:00:00,2011-03-28 00:00:00,0,0.000000,0.000000,54.96,0.000000,0,0,0,Thursday,BOM
4,8V726M,1,2010-11-27,2010-11-29 00:00:00,2013-01-28 00:00:00,30,90.000000,13.333333,111.91,0.008850,0,0,0,Monday,BOM


In [ ]:
#2) Start SageMaker Canvas (AWS Console)

#Go to Amazon SageMaker → Canvas. - Jaffer Done


#SageMaker → Domains → Create domain (Studio setup). - Jaffer Done

#Create a user profile with Canvas enabled. - Jaffer Done

#In Canvas, Create new dataset → pick Local upload or S3 and point to your dataset.csv. - Jaffer Done

#Tip: If the file is bigger than 5GB, upload to S3 and point Canvas to it. - Jaffer Done
 

In [ ]:
#3) Open Data Wrangler and upload the dataset - Jaffer Done

#In SageMaker Studio, open your user profile. - Jaffer Done

#Left pane: Data → Data Wrangler → Create data flow. - Jaffer Done

#Import → Upload file (local) → choose dataset.csv.  
#(Alternatively: Import → S3 if you placed it in S3.)  - Jaffer Done

In [ ]:
#4) Explore, clean, and engineer features (Data Wrangler)

#In your Data Flow:

#Preview → review column types; fix incorrect types (dates, bools, numerics).

#Handle missing values: drop, fill (median/most frequent), or flag with indicator columns.






In [18]:


#Feature engineering ideas (be creative but reproducible):

import pandas as pd
import numpy as np

# Data Wrangler calls this function. `df` is a pandas DataFrame of the current step.
def transform(df: pd.DataFrame) -> pd.DataFrame:
    # --- Ensure date types ---
    for col in ["created", "firstorder", "lastorder"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")

    # Helper: days between (safe)
    def days_between(later, earlier):
        delta = (later - earlier).dt.total_seconds() / (24 * 3600)
        return pd.to_numeric(delta, errors="coerce")

    # 1) customer_age_days = today - created
    today = pd.Timestamp.utcnow().normalize()
    df["customer_age_days"] = days_between(pd.Series(today, index=df.index), df["created"])

    # 2) tenure_days = lastorder - firstorder
    if "lastorder" in df.columns and "firstorder" in df.columns:
        df["tenure_days"] = days_between(df["lastorder"], df["firstorder"])

    # 3) avg_click_rate = eclickrate / eopenrate (safe)
    if "eclickrate" in df.columns and "eopenrate" in df.columns:
        den = df["eopenrate"].replace(0, np.nan)
        df["avg_click_rate"] = df["eclickrate"] / den

    # 4) is_loyal_customer = (ordfreq > 0.10)
    if "ordfreq" in df.columns:
        df["is_loyal_customer"] = (df["ordfreq"] > 0.10).astype(int)

    # 5) email_engagement = (eopenrate + eclickrate) / 2
    if "eopenrate" in df.columns and "eclickrate" in df.columns:
        df["email_engagement"] = (df["eopenrate"] + df["eclickrate"]) / 2.0

    # 6) high_spender_flag = (avgorder > 100)
    if "avgorder" in df.columns:
        df["high_spender_flag"] = (df["avgorder"] > 100).astype(int)

    # 7) order_per_day = ordfreq / tenure_days (safe)
    if "ordfreq" in df.columns and "tenure_days" in df.columns:
        tenure_safe = df["tenure_days"].where(df["tenure_days"] > 0, np.nan)
        df["order_per_day"] = df["ordfreq"] / tenure_safe

    # 8) is_paperless_user = (paperless == 1)
    if "paperless" in df.columns:
        df["is_paperless_user"] = (df["paperless"] == 1).astype(int)

    # 9) One-hot encode favday and city (optional; use built-ins if you prefer)
    if "favday" in df.columns:
        fav_dummies = pd.get_dummies(df["favday"], prefix="favday", dummy_na=True)
        df = pd.concat([df.drop(columns=["favday"]), fav_dummies], axis=1)

    if "city" in df.columns:
        city_dummies = pd.get_dummies(df["city"], prefix="city", dummy_na=True)
        df = pd.concat([df.drop(columns=["city"]), city_dummies], axis=1)

    # Clean infinities
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    return df

    
#When satisfied:

#Top right: Export → Jupyter Notebook (and .flow file).
#Choose the S3 location for export artifacts if prompted.

#You will get:

 A Jupyter .ipynb with all transforms as PySpark or Pandas (you pick during export).

#A Data Wrangler flow file (*.flow) describing the pipeline.

In [ ]:
#5) In SageMaker Studio: run the exported notebook → export to SageMaker Feature Store

#In Studio, Upload the exported .ipynb and .flow.

#Open the .ipynb and Run All.

#Confirm the final cells write to Feature Store:

#The export usually includes code to create a Feature Group and ingest the processed features.

#If your export targets S3 instead, add a short cell to ingest to Feature Store (sample below).

#Example: Feature Store ingestion (if needed)